In [1]:
import numpy as np
np.random.seed(42)
import faiss


Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


In [2]:
x = np.random.uniform(-1,10,[1500,300]).astype(np.float32)
y = np.random.uniform(-1,10,[200000,300]).astype(np.float32)

In [3]:
x.dtype

dtype('float32')

In [4]:
%%time
def get_top10_vectors(vector,matrix):
    unsorted = ((np.matmul(vector,matrix.T)/(np.linalg.norm(vector)*np.sqrt(np.einsum('ij,ij->i',matrix, matrix)))))
    #print(unsorted[:2])
    unsorted = np.argsort(unsorted,kind="quicksort")
    distances = unsorted[::-1][:10]
    del unsorted
    return distances

tmp= get_top10_vectors(x[0],y)
print(tmp)

[ 58744 161973  34627  28188  72988 191155 188798  72243 195993 140433]
CPU times: user 165 ms, sys: 266 ms, total: 431 ms
Wall time: 269 ms


In [5]:
index = faiss.IndexFlatL2(x.shape[1])
print(index.is_trained)
index.add(y) # vectors db
k = 10                    # we want to see 4 nearest neighbors

True


In [6]:
%time
D, I = index.search(x, k) # sanity check

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.4 µs


In [7]:
I[0]

array([ 58744,  34627, 191155, 178936, 168397, 176887, 185882, 123922,
       162680, 105041])

In [8]:
import nmslib

In [10]:
%%time
# initialize a new index, using a HNSW index on Cosine Similarity
#{'angular': 'cosinesimil', 'euclidean': 'l2'}[metric]
index = nmslib.init(method='hnsw', space='l2')
index.addDataPointBatch(y)
index.createIndex({'post': 2}, print_progress=True)

CPU times: user 32min 8s, sys: 2.84 s, total: 32min 11s
Wall time: 8min 41s


In [11]:
tmp

array([ 58744, 161973,  34627,  28188,  72988, 191155, 188798,  72243,
       195993, 140433])

In [12]:
# query for the nearest neighbours of the first datapoint
ids, distances = index.knnQuery(x[0], k=10)

In [13]:
ids

array([121020,  66934,  70276, 125972,   4032,  28699, 156761,  85153,
         4234, 145234], dtype=int32)